# Agent Based Model: Main Notebook

In [51]:
import numpy.random as rnd
import pandas as pd 
from matplotlib import pyplot as plt

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Creating Study Groups

#### Practical Functions

In [52]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(2.5, 2.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000)

In [53]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    ID_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []

    for student in studygroup:
        ID_list.append(student.ID)
        extraversion_list.append(student.Extraversion())
        sensing_list.append(student.Sensing()) 
        thinking_list.append(student.Thinking())
        judging_list.append(student.Judging())
    
    data = pd.DataFrame({'ID': ID_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list})

    return data

#### Create Study Group

In [54]:
## Defining the Students ##
class Student():
    def __init__(self, ID, Ex, Se, Th, Ju):
        self.ID  = ID
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension

    def Extraversion(self):
        if self.Ex == "E":
            self.ExScore = max_vals[0]
            del max_vals[0]
            return self.ExScore
        else:
            self.ExScore = min_vals[0]
            del min_vals[0]
            return self.ExScore

    def Sensing(self):
        if self.Se == "S":
            self.SeScore = max_vals[0]
            del max_vals[0]
            return self.SeScore
        else:
            self.SeScore = min_vals[0]
            del min_vals[0]
            return self.SeScore
    
    def Thinking(self):
        if self.Th == "T":
            self.ThScore = max_vals[0]
            del max_vals[0]
            return self.ThScore
        else: 
            self.ThScore = min_vals[0]
            del min_vals[0]
            return self.ThScore
    
    def Judging(self):
        if self.Ju == "J":
            self.JuScore = max_vals[0]
            del max_vals[0]
            return self.JuScore
        else: 
            self.ThScore = min_vals[0]
            del min_vals[0]
            return self.ThScore

In [55]:
def StudyGroup(student_list):
    studygroup = []

    for i in range(len(student_list)):
        student = Student(i+1, student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        studygroup.append(student)
    
    return data_collect(studygroup, student_list)

In [56]:
student_list_test = ["ESTJ", "ISTJ", "ENTP", "INFJ"]

studygroup1 = StudyGroup(student_list_test)

print(studygroup1)


   ID  type       E/I       S/N       T/F       J/P
0   1  ESTJ  0.542167  0.692618  0.776069  0.575923
1   2  ISTJ  0.276957  0.847295  0.722213  0.626004
2   3  ENTP  0.584871  0.468179  0.753029  0.109836
3   4  INFJ  0.305042  0.473969  0.160686  0.554537


In [57]:
student_list_test_2 = ["ESTJ", "ISTJ", "ENTP", "INFJ"]

studygroup2 = StudyGroup(student_list_test_2)

print(studygroup2)

   ID  type       E/I       S/N       T/F       J/P
0   1  ESTJ  0.645753  0.846695  0.839897  0.852751
1   2  ISTJ  0.345219  0.866033  0.807983  0.543229
2   3  ENTP  0.864734  0.293909  0.828453  0.128762
3   4  INFJ  0.295856  0.473946  0.293002  0.700003


In [59]:
student_list_test_3 = ["ESTJ", "ENTJ", "ENTP", "ESTP"]

studygroup3 = StudyGroup(student_list_test_3)

print(studygroup3)

   ID  type       E/I       S/N       T/F       J/P
0   1  ESTJ  0.531731  0.737082  0.734511  0.681113
1   2  ENTJ  0.701183  0.151122  0.713991  0.830095
2   3  ENTP  0.838384  0.295055  0.504167  0.228464
3   4  ESTP  0.561616  0.590703  0.502538  0.177439


## Creating the ABM